## Машинное обучение, ВМК МГУ

## Практическое задание 2

### Общая информация
Дата выдачи: 9 октября 2019

Максимальная оценка: 10 баллов + 1 бонусный балл

Мягкий дедлайн: 23:59MSK 23 октября (за каждый день просрочки снимается 1 балл)

Жесткий дедлайн: 23:59MSK 30 октября.

### О задании

В этом задании вы:
- Познакомитесь с методом решения задачи регрессии на основе метода ближайших соседей.
- Реализуете алгоритм kNN для задачи регрессии.
- Изучите методы работы с категориальными и текстовыми переменными.


### Оценивание и штрафы
Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимально допустимая оценка за работу — 10 баллов.

Сдавать задание после указанного срока сдачи нельзя. При выставлении неполного балла за задание в связи с наличием ошибок на усмотрение проверяющего предусмотрена возможность исправить работу на указанных в ответном письме условиях.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов (подробнее о плагиате см. на странице курса). Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

Неэффективная реализация кода может негативно отразиться на оценке.


### Формат сдачи
Для сдачи задания переименуйте получившийся файл *.ipynb в соответствии со следующим форматом: homework-practice-02-Username.ipynb, где Username — ваша фамилия и имя на латинице именно в таком порядке (например, homework-practice-02-ivanov.ipynb).

Далее отправьте этот файл на anytask в соответсвующий раздел.

In [1]:
import numpy as np
import pandas as pd

Все эксперименты в этой лабораторной работе предлагается проводить на данных соревнования New York City Airbnb Open Data: https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data#AB_NYC_2019.csv

В данной задаче предлагается предсказать цену на съем квартиры в зависимости от её параметров.

In [9]:
data = pd.read_csv('./data/AB_NYC_2019.csv')
data.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [10]:
data.shape

(48895, 16)

In [11]:
# число значений у признаков
for col_name in data.columns:
    print(col_name, len(data[col_name].unique()))

id 48895
name 47906
host_id 37457
host_name 11453
neighbourhood_group 5
neighbourhood 221
latitude 19048
longitude 14718
room_type 3
price 674
minimum_nights 109
number_of_reviews 394
last_review 1765
reviews_per_month 938
calculated_host_listings_count 47
availability_365 366


In [12]:
data.isna().sum()

id                                    0
name                                 16
host_id                               0
host_name                            21
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                       10052
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64

In [13]:
data

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48890,36484665,Charming one bedroom - newly renovated rowhouse,8232441,Sabrina,Brooklyn,Bedford-Stuyvesant,40.67853,-73.94995,Private room,70,2,0,NaN,NaN,2,9
48891,36485057,Affordable room in Bushwick/East Williamsburg,6570630,Marisol,Brooklyn,Bushwick,40.70184,-73.93317,Private room,40,4,0,NaN,NaN,2,36
48892,36485431,Sunny Studio at Historical Neighborhood,23492952,Ilgar & Aysel,Manhattan,Harlem,40.81475,-73.94867,Entire home/apt,115,10,0,NaN,NaN,1,27
48893,36485609,43rd St. Time Square-cozy single bed,30985759,Taz,Manhattan,Hell's Kitchen,40.75751,-73.99112,Shared room,55,1,0,NaN,NaN,6,2


Как видите, в данных есть пропуски. Не забудьте обработать их

Заполним пропуски в колонках **name** и **host_name** значением '-1', которое будет сигнализировать о пропуске в колонке.

In [14]:
data.loc[data['name'].isna(), 'name'] = '-1'
data.loc[data['host_name'].isna(), 'host_name'] = '-1'

С колонкой $last\_review$ поступим иначе:
* $last\_review$ - рассмотрим объект $x$ такой, что $x["last\_review"] = nan$ и $x["neighbourhood"] = neigh$. Тогда $x["last\_review"]$ заполняем наиболее встречаемым значением в колонке $last\_review$ среди всех объектов $x_{i}$ тренировочной выборки: $x_{i}["neighbourhood"] = neigh$

In [15]:
%%time
is_last_review_nan = data['last_review'].isna()
for neigh in data[is_last_review_nan]['neighbourhood']:
    is_neigh = data['neighbourhood'] == neigh
    if data[is_neigh].shape[0] == 1:
        data.loc[is_last_review_nan, 'last_review'] = '-1'
    else:
        data.loc[is_neigh & is_last_review_nan, 'last_review'] = data[is_neigh]['last_review'].value_counts().idxmax()

CPU times: user 1min 39s, sys: 35.7 ms, total: 1min 39s
Wall time: 1min 39s


А для колонки **$reviews\_per\_month$** возьмем не наиболее встречаемое значение, а среднее:

In [16]:
%%time
is_reviews_per_month_nan = data['reviews_per_month'].isna()
for neigh in data[is_last_review_nan]['neighbourhood']:
    is_neigh = data['neighbourhood'] == neigh
    if data[is_neigh].shape[0] == 1:
        data.loc[is_reviews_per_month_nan, 'reviews_per_month'] = -1
    else:
        data.loc[is_neigh & is_reviews_per_month_nan, 'reviews_per_month'] = data[is_neigh]['reviews_per_month'].mean()

CPU times: user 1min 2s, sys: 28 ms, total: 1min 2s
Wall time: 1min 2s


Разобъем данные на обучение и контроль.

Для тестирования модели без категориальных признаков:

In [17]:
from sklearn.model_selection import train_test_split
X_train_non_cat, X_test_non_cat, y_train_non_cat, y_test_non_cat = train_test_split(data.drop(columns=['price',
                                                                                              'last_review',
                                                                                              'name',
                                                                                              'host_name',
                                                                                              'neighbourhood_group',
                                                                                              'neighbourhood',
                                                                                              'room_type']),
                                                                                    data[['price']],
                                                                                    test_size=0.3, random_state=241)

Для тестирования только категориальных признаков(wihtout 'name'):

In [18]:
from sklearn.model_selection import train_test_split
X_train_cat, X_test_cat, y_train_cat, y_test_cat = train_test_split(data.drop(columns=['price',
                                                                                       'id',
                                                                                       'name',
                                                                                       'host_id',
                                                                                       'latitude',
                                                                                       'longitude',
                                                                                       'minimum_nights',
                                                                                       'number_of_reviews',
                                                                                       'reviews_per_month',
                                                                                       'calculated_host_listings_count',
                                                                                       'availability_365']),
                                                                                    data[['price']],
                                                                                    test_size=0.3, random_state=241)

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.drop(columns=['price']),
                                                    data[['price']],
                                                    test_size=0.3, random_state=241)

## Часть 1: Алгоритм kNN в задаче регрессии

<b> 1.1 (1.5 балла) </b>
Реализуйте класс `KNNRegressor`, который используя метод k ближайших соседей решает задачу регрессии. Для решение данной задачи, необходимо найти $N_k$ - k соседей, и после использовать значения их целевых переменных для предсказания:
\begin{align}
y = \frac{1}{k}\sum_{n \in N_k}w_n y_n,
\end{align}

где $w_n$ - вес каждого соседа. 

При этом `KNNRegressor` может работать в 2 режимах:
 - $uniform$ - ближайшие соседи учитываются с одинаковыми весами.
 - $distance$ - вес ближайших соседей зависит от расстояния
 
Сигнатуру методов при желании можно менять.

In [401]:
from typing import Union, Callable, Iterable, Optional
from sklearn.neighbors import NearestNeighbors


class KNNRegressor:
    def __init__(self, n_neighbors: int,
                 metric: Union[str, Callable],
                 mode: str = 'uniform',
                test_block_size = 10):
        """
        params:
            n_neighbors: number of neighbors
            metric: metric to use for distance computation
            mode: 'uniform' or 'distance'
            'uniform' - all points in each neighborhood are weighted equally
            'distance' - weight points by the inverse of their distance
            test_block_size: block size for finding neighbors and predict (for saving memory)
        """
        
        self.metric = metric
        self.__nn = None
        if self.metric not in ['overlap', 'flattened_overlap', 'log_overlap']:
            self.__nn = NearestNeighbors(n_neighbors=n_neighbors, metric=self.metric)
        self.__mode = mode
        self.n_neighbors = n_neighbors
        self.test_block_size = test_block_size
        self.y_train = None
        self.neigh_idxs = None
        self.distances = None

    def fit(self, X: np.array, y: np.array) -> None:
        """
            X: data
            y: labels
        """
        
        self.y_train = y
        if self.__nn is None:
            self.X_train = X
        else:
            self.__nn.fit(X, y)
        self.eps = 1e-5
        
    def flattened_overlap_df(self, x_df, z_df):
        result = np.zeros(x_df.shape[0], z_df.shape[0])
        for i, x_row in enumerate(x_df):
            result[i, :] = np.apply_along_axis(lambda z: flattened_overlap(x_row, z), axis=1)
        return result
    
        
    def find_kneighbors(self, X, return_distance):
        """
        params:
            * X - objects sample
            * return_distance - bool variable

        return values:
            * If return_distance == True:
                * tuple with two numpy array with size (X.shape[0], k), where:
                  [i, j] elem of first array must be the distance between
                  i-th object and his j-th nearest neighbour
                  [i, j] elem of second array must be the index of j-th nearest neighbour to i-th object
            * If return_distance == False:
                * only second array
        """


        if self.__nn is not None:
            self.distances, \
                self.neigh_idxs = self.__nn.kneighbors(X, n_neighbors=self.n_neighbors)
        else:
            if self.metric == 'overlap':
                self.distances = overlap(X, self.X_train)
            elif self.metric == 'flattened_overlap':
                self.distances = flattened_overlap_df(X, self.X_train, self.X_train)
            elif self.metric == 'log_overlap':
                self.distances = log_overlap_df(X, self.X_train, X, self.X_train)
            self.neigh_idxs = np.argsort(self.distances,
                                         axis=1)[:, :self.n_neighbors]
            if return_distance:
                self.distances = self.distances[np.arange(self.distances.shape[0])[:, None],
                                                self.neigh_idxs]
        if return_distance:
            return self.distances, self.neigh_idxs
        return self.neigh_idxs

    def predict(self, X: np.array, n_neighbors: Optional[int] = None) -> np.array:
        """
            X: data
            n_neighbors: number of neighbors
        """
        if n_neighbors is not None:
            n_neighbors = np.minimum(self.n_neighbors, n_neighbors)
        else:
            n_neighbors = self.n_neighbors
        if self.test_block_size > X.shape[0]:
            self.test_block_size = X.shape[0]
        preds = np.zeros(X.shape[0])
        split_size = X.shape[0] // self.test_block_size + \
                     int(X.shape[0] % self.test_block_size != 0)
        curr_idx = 0
        for i, split in enumerate(np.array_split(X, split_size)):
            self.distances, self.neigh_idxs = self.find_kneighbors(split,
                                                                   return_distance=True
                                                                  )
            for j, idx in enumerate(self.neigh_idxs):
                if self.__mode == 'distance':
                    weights = 1 / (self.distances[j] + self.eps)
                    pred = np.sum(self.y_train.values[idx].reshape(-1) * weights) / np.sum(weights)
                else:
                    pred = np.sum(self.y_train.values[idx].reshape(-1)) / n_neighbors
                preds[j + curr_idx] = pred
            curr_idx += split.shape[0]
        return preds
    
    
    def predict_without_testblock(self, X):
        """
        params:
            * X - test objects

        return values:

        """
        
        self.distances, self.neigh_idxs = self.find_kneighbors(X,
                                                               return_distance=True
                                                              )
        preds = np.zeros(X.shape[0])
        for j, idx in enumerate(self.neigh_idxs):
            if self.__mode == 'distance':
                weights = 1 / (self.distances[j] + self.eps)
                pred = np.sum(self.y_train[idx] * weights) / np.sum(weights)
            else:
                pred = np.sum(self.y_train[idx]) / self.n_neighbors
            preds[j] = pred
        return preds
    
    
    def predict_opt(self, X):
        """
        params:
            * X - test objects

        return values:

        """

        preds = np.zeros(X.shape[0])
        for j, idx in enumerate(self.neigh_idxs[:, :self.n_neighbors]):
            if self.__mode == 'distance':
                weights = 1 / (self.distances[j, :self.n_neighbors] + self.eps)
                counts = np.bincount(self.y_train[idx],  weights)
            else:
                counts = np.bincount(self.y_train[idx])
            preds[j] = np.argmax(counts)
        return preds
    
    
    def predict_opt_regression(self, X):
        """
        params:
            * X - test objects

        return values:

        """

        preds = np.zeros(X.shape[0])
        for j, idx in enumerate(self.neigh_idxs[:, self.n_neighbors]):
            if self.__mode == 'distance':
                weights = 1 / (self.distances[j, self.n_neighbors] + self.eps)
                pred = np.sum(self.y_train[idx] * weights) / np.sum(weights)
            else:
                pred = np.sum(self.y_train[idx]) / self.n_neighbors
            preds[j] = pred
        return preds

In [21]:
def rmse(y_true, y_valid):
    return (np.sum((y_valid - y_true) ** 2) / len(y_true)) ** (1/2)

In [22]:
k_neighbors = 10
knn = KNNRegressor(k_neighbors, metric='euclidean', mode='euclidean', test_block_size=500)

In [23]:
%%time
knn.fit(X_train_non_cat, y_train_non_cat)
preds = knn.predict(X_test_non_cat, 10)

CPU times: user 388 ms, sys: 0 ns, total: 388 ms
Wall time: 388 ms


In [24]:
rmse(y_test_non_cat.values.reshape(-1), preds)

224.97411992865153

## Часть 2: Категориальные признаки

<b>2.1 (1 балл)</b>
Реализуйте три функции расстояния на категориальных признаках, которые обсуждались на [третьем семинаре](https://github.com/mmp-mmro-team/mmp_mmro_fall_2019/blob/master/lecture-notes/Sem03_knn.pdf). Не забудьте, что KNNRegressor должен уметь работать с этими функциями расстояния. Как вариант, можно реализовать метрики как [user-defined distance](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html).

In [25]:
# import pyximport; pyximport.install() Попытки переписать метрики на Cython оказались безуспешными
# import overlap_cython

In [26]:
data_cat = data.copy()
for column in data.columns:
    if data_cat[column].dtype != 'O':
        data_cat.drop(columns=column, axis=1, inplace=True)

Уберем 'name', так как этот признак не является информативным:

In [27]:
data_cat.drop('name', axis=1, inplace=True)

Посчитаем статистику для всех данных для ускорения работы метрик

In [28]:
from tqdm import tqdm_notebook

In [29]:
from sklearn.preprocessing import LabelEncoder

In [30]:
le = LabelEncoder()
for column in data_cat.columns:
    all_classes_in_col = data_cat[column].unique()
    le.fit(all_classes_in_col)
    data_cat.loc[:, column] = le.transform(data_cat.loc[:, column])

In [31]:
%%time
f_stat = np.zeros((data_cat.shape[0], data_cat.shape[1]))
p2_sum_stat = np.zeros((data_cat.shape[0], data_cat.shape[1]))
data_cat_np = data_cat.to_numpy(copy=True)

for i, column in enumerate(data_cat_np.T):
    print(column)
    is_equal = column[:, None] == column
    f_stat[:, i] = np.sum(is_equal, axis=0)

[ 4990  4786  2910 ...  4264 10191  1969]
[1 2 2 ... 2 2 2]
[108 127  94 ...  94  95  95]
[1 0 1 ... 0 2 1]
[1502 1716    0 ...    0    0    0]
CPU times: user 11 s, sys: 2.96 s, total: 14 s
Wall time: 14 s


In [32]:
p2_sum_arr = np.zeros(data_cat.shape[1])
for i, line in enumerate(tqdm_notebook(data_cat_np)):
    for j, x_val in enumerate(line):
        f_x = data_cat.iloc[:, j].value_counts()
        p2 = f_x
        p2 = p2 * (p2 - 1) / (data_cat.shape[0] * (data_cat.shape[0] - 1))
        p2_sum_arr[j] = np.sum(p2 * (f_x.values <= f_x[x_val]))
    p2_sum_stat[i, :] = p2_sum_arr

KeyboardInterrupt: 

In [33]:
def overlap(x, z):
    """
    x, z - pandas dataframes with shape lxd
    """
    x = x.to_numpy(copy=True)
    z = z.to_numpy(copy=True)
    is_equal = x[:, None] != z
    return np.sum(is_equal, axis=2)

def flattened_overlap(x, z, df):
    row_idx = np.sum(df == x, axis=1).idxmax()
    result = np.sum(x != z) + np.sum((x == z) * p2_sum_stat[row_idx, :])
    return result


def log_overlap(x, z, df_x, df_z):
    result = 0
    row_idx_x = np.sum(df_x == x, axis=1).idxmax()
    row_idx_z = np.sum(df_z == z, axis=1).idxmax()
    result = np.sum((x != z) * np.log(f_stat[row_idx_x] + 1) * np.log(f_stat[row_idx_z] + 1))
    return result

In [34]:
%%time
flattened_overlap(X_train_cat.iloc[7, 1:],
                  X_test_cat.iloc[0, 1:],
                  data_cat)
            #data.loc[:, ['host_name', 'neigbhourhood_group', 'neighbourhood', 'room_type', 'last_review']])

/home/paniquex/anaconda3/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


ValueError: operands could not be broadcast together with shapes (4,) (5,) 

In [50]:
%%time
log_overlap(X_train_cat.iloc[0, 1:],# 'last_review']],
            X_train_cat.iloc[2, 1:],# 'last_review']],
            data_cat,
            data_cat)

CPU times: user 15.3 ms, sys: 0 ns, total: 15.3 ms
Wall time: 14.7 ms


174.28139809655437

In [35]:
k_neighbors = 10
knn = KNNRegressor(k_neighbors, metric='overlap', mode='distance', test_block_size=500)

In [36]:
%%time
knn.fit(X_train_cat.iloc[:, :], y_train_cat.iloc[:, :])
preds = knn.predict(X_test_cat.iloc[:, :], 10)

CPU times: user 46.1 s, sys: 2.84 s, total: 48.9 s
Wall time: 48.9 s


In [37]:
rmse(y_test_non_cat.values.reshape(-1), preds)

235.35793736181222

In [54]:
#228.2563985959649 # flattened 
#227.45479221155136 # log_overlap
#222.72230579804977 # overlap

## Воспользуемся LabelEncoder для того, чтобы преобразовать категориальные признаки

In [38]:
from sklearn.preprocessing import LabelEncoder

In [39]:
le = LabelEncoder()
for column in X_train_cat.columns:
    all_classes_in_col = np.hstack([X_train_cat[column].unique(), X_test_cat[column].unique()])
    le.fit(all_classes_in_col)
    X_train_cat.loc[:, column] = le.transform(X_train_cat.loc[:, column])
    X_test_cat.loc[:, column] = le.transform(X_test_cat.loc[:, column])


In [40]:
def flattened_overlap_df(x_df, z_df, df):
        result = np.zeros((x_df.shape[0], z_df.shape[0]))
        for i, x_row in enumerate(x_df.iterrows()):
            print(i)
            x_row = np.array(x_row[1])
            result[i, :] = np.apply_along_axis(lambda z: flattened_overlap(x_row, z, df), axis=1, arr=z_df)
        return result
    
def log_overlap_df(x_df, z_df, full_df_x, full_df_z):
        result = np.zeros((x_df.shape[0], z_df.shape[0]))
        for i, x_row in enumerate(x_df.iterrows()):
            x_row = np.array(x_row[1])
            result[i, :] = np.apply_along_axis(lambda z: log_overlap(x_row, z,
                                                                     full_df_x, full_df_z),
                                               axis=1,
                                               arr=z_df)
            print(i)
        return result

In [85]:
%%time
log_overlap_distances = log_overlap_df(X_test_cat.iloc[:50, 1:], X_train_cat.iloc[:5000, 1:],
                                       X_test_cat.iloc[:50, 1:], X_train_cat.iloc[:5000, 1:])

KeyboardInterrupt: 

In [ ]:
%%time
flattened_overlap_distances = flattened_overlap_df(X_test_cat.iloc[:, 1:], X_train_cat.iloc[:, 1:], data_cat)

0
1
2
3
4
5


In [97]:
flattened_overlap_distances[0, 7]

3.6621321218146297

In [76]:
knn = KNNRegressor(10, metric='overlap', mode='uniform', test_block_size=1000)
test_size =  100 #X_train_cat.shape[0]
knn.fit(X_train_cat[:20000], y_train_cat[:20000])

In [77]:
%%time
preds = knn.predict(X_test_cat[:test_size])

CPU times: user 77.9 ms, sys: 4.07 ms, total: 82 ms
Wall time: 79.8 ms


In [78]:
rmse(y_test_cat.iloc[:test_size].values.reshape(-1), preds)

170.4348294803618

In [74]:
knn = KNNRegressor(10, metric='flattened_overlap', mode='uniform', test_block_size=10)
knn.fit(X_train_cat.iloc[:5000, 1:], y_train_cat[:5000])

In [75]:
%%time
preds = knn.predict(X_test_cat.iloc[:test_size, 1:])

0
0
1
1
2
2
3
3
4
4
5
5
6
6
7
7
8
8
9
9
CPU times: user 1min 21s, sys: 0 ns, total: 1min 21s
Wall time: 1min 21s


In [77]:
rmse(y_test_cat.values.reshape(-1)[:test_size], preds)

38.41189919803497

In [289]:
knn = KNNRegressor(10, metric='log_overlap', mode='uniform', test_block_size=100)
knn.fit(X_train_cat.iloc[:20, 1:], y_train_cat.iloc[:20, 1:])

In [290]:
%%time
preds = knn.predict(X_test_cat.iloc[:10, 1:])

0
1
2
3
4
5
6
7
8
9
CPU times: user 90.6 ms, sys: 3.96 ms, total: 94.5 ms
Wall time: 92.5 ms


In [76]:
rmse(y_test_cat.values.reshape(-1)[:10], preds)

38.41189919803497

<b>2.2 (1 балл)</b> Найдите все категориальные признаки в данных. Подсчитайте для каждой из метрик качество на тестовой выборке `X_test` при числе соседей $k = 10$. Качество измеряйте с помощью RMSE.

Какая функция расстояния оказалась лучшей? Почему?

In [31]:
# Ваш код здесь

<b>2.3 (1 балл) бонус</b> Подберите лучшее (на тестовой выборке) число соседей $k$ для каждой из функций расстояния. Какого удалось достичь уровня качества?

In [32]:
# Ваш код здесь

<b>2.4 (2.5 балла)</b> Отойдем ненадолго от задачи регрессии и перейдём к задаче классификации: будем определять, являеться ли квартира дорогой $(target = 1)$ или дешевой $(target = 0)$. Будем считать дорогими квариры, цена которых выше среднего.

In [85]:
data['target'] = (data.price > data.price.mean()).astype(int)

Реализуйте счетчики, которые заменят категориальные признаки на вещественные.

А именно, для каждого категориального признака $f_j(x)$ необходимо сделать следующее:
1. Число `counts` объектов в обучающей выборке с таким же значением признака.
\begin{align}
counts_j(c) = \sum_{i=1}^l [f_j(x_i) = c]
\end{align}
2. Число `successes` объектов первого класса ($y = 1$) в обучающей выборке с таким же значением признака.
\begin{align}
successes_j(c) = \sum_{i=1}^l[f_j(x_i) = c][y_i = +1].
\end{align}
3. Сглаженное отношение двух предыдущих величин:
\begin{align}
p_j(c) = \frac{successes_j(c) + a}{counts_j(c) + b},
\end{align}

где $a$ и $b$ - априорные счетчики (например, a = 1, b = 2).

In [310]:
def counts(x, x_tr):
    """
    x - one categorical column with type - pandas series
    x_tr
    """
    
    x = x.to_numpy()
    x_tr = x_tr.to_numpy()
    return np.sum(x[:, None] == x_tr, axis=1)


def successes(x, x_tr, y):
    """
    x - one categorical column with type - pandas series
    y - target feature from df
    """
    
    x = x.to_numpy()
    x_tr = x_tr.to_numpy()
    y = y.to_numpy()
    print(x.shape, x_tr.shape)
    result = np.sum((x[:, None] == x_tr) * (y == 1), axis=1)
    return result
    

def flattened_c_s(x, x_tr, y, a=1, b=2):
    """
    x - one categorical column with type - pandas series
    y - target feature from dataframe
    a - priori counter
    b - priori counter
    """
    
    return (successes(x, x_tr, y) + a) / (counts(x, x_tr) + b)

In [292]:
%%time
cnts = counts(X_test_cat.iloc[:, 0], X_train_cat.iloc[:, 0])

(14669,)
(34226,)
CPU times: user 847 ms, sys: 301 ms, total: 1.15 s
Wall time: 1.31 s


In [168]:
cnts[:50]

array([ 35, 164,  34,  78,  19,  69,  31,  10, 164,   7, 104,   2,   0,
        32,   0,   0,   7,   1,   6,  60, 105, 278,  81,   0,   7,   3,
        25,  17,  58,   9,  17,   0,   1,   0,   1,   7,   1,  59,  77,
        79,  13,  26, 103,   2, 164,  61,  18,   0,  16,   2])

In [195]:
%%time
scss = successes(X_test_cat['neighbourhood_group'][:], X_train_cat['neighbourhood_group'][:], y_test_cat)

(14669,) (34226,)
CPU times: user 1.09 s, sys: 249 ms, total: 1.34 s
Wall time: 1.34 s


In [170]:
%%time
# flattened_c_s(data['neighbourhood_group'][:10], data['target'][:10])

CPU times: user 14 µs, sys: 1e+03 ns, total: 15 µs
Wall time: 29.6 µs


In [171]:
def counters(df_cat, X_train, y):
    df_copy = df_cat.copy()
    for column in df_copy.columns:
        print(column)
        df_copy[column + '_counts'] = counts(df_copy[column], X_train[column])
        df_copy[column + '_successes'] = successes(df_copy[column], X_train[column], y)
        df_copy[column + '_flattened_c_s'] = flattened_c_s(df_copy[column], X_train[column], y)
        df_copy.drop(column, axis=1, inplace=True)
    return df_copy

In [172]:
from sklearn.model_selection import train_test_split
X_train_cat, X_test_cat, y_train_cat, y_test_cat = train_test_split(data.drop(columns=['price',
                                                                                       'target',
                                                                                        'id',
                                                                                        'name',
                                                                                        'host_id',
                                                                                        'latitude',
                                                                                        'longitude',
                                                                                        'minimum_nights',
                                                                                        'number_of_reviews',
                                                                                        'reviews_per_month',
                                                                                        'calculated_host_listings_count',
                                                                                        'availability_365']),
                                                                                    data[['target']],
                                                                                    test_size=0.3, random_state=241)

In [173]:
le = LabelEncoder()
for column in X_train_cat.columns:
    all_classes_in_col = np.hstack([X_train_cat[column].unique(), X_test_cat[column].unique()])
    le.fit(all_classes_in_col)
    X_train_cat.loc[:, column] = le.transform(X_train_cat.loc[:, column])
    X_test_cat.loc[:, column] = le.transform(X_test_cat.loc[:, column])

In [175]:
%%time
counters(X_test_cat, X_train_cat, y_test)

host_name
(14669,) (34226,)
(14669,) (34226,)
neighbourhood_group
(14669,) (34226,)
(14669,) (34226,)
neighbourhood
(14669,) (34226,)
(14669,) (34226,)
room_type
(14669,) (34226,)
(14669,) (34226,)
last_review
(14669,) (34226,)
(14669,) (34226,)
CPU times: user 16.5 s, sys: 4.01 s, total: 20.5 s
Wall time: 20.5 s


,host_name_counts,host_name_successes,host_name_flattened_c_s,neighbourhood_group_counts,neighbourhood_group_successes,neighbourhood_group_flattened_c_s,neighbourhood_counts,neighbourhood_successes,neighbourhood_flattened_c_s,room_type_counts,room_type_successes,room_type_flattened_c_s,last_review_counts,last_review_successes,last_review_flattened_c_s
2850,35,0,0.027027,14061,0,0.000071,768,0,0.001299,17846,0,0.000056,6,0,0.125000
44256,164,0,0.006024,15179,0,0.000066,804,0,0.001241,17846,0,0.000056,6949,0,0.000144
6636,34,0,0.027778,14061,0,0.000071,2748,0,0.000364,17846,0,0.000056,4,0,0.166667
332,78,0,0.012500,14061,0,0.000071,104,0,0.009434,17846,0,0.000056,955,0,0.001045
43143,19,0,0.047619,15179,0,0.000066,1387,0,0.000720,17846,0,0.000056,86,0,0.011364
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3880,2,0,0.250000,15179,0,0.000066,774,0,0.001289,17846,0,0.000056,949,0,0.001052
48120,55,0,0.017544,15179,0,0.000066,257,0,0.003861,17846,0,0.000056,6949,0,0.000144
456,114,0,0.008621,15179,0,0.000066,1281,0,0.000779,17846,0,0.000056,626,0,0.001592
42059,60,0,0.016129,14061,0,0.000071,48,0,0.020000,15563,0,0.000064,298,0,0.003333


Поскольку признаки, содержащие информацию о целевой переменной, могут привести к переобучению, может оказаться полезным сделать *фолдинг*: разбить обучающую выборку на $n$ частей, и для $i$-й части считать `counts` и `successes` по всем остальным частям. Для тестовой выборки используются счетчики, посчитанный по всей обучающей выборке. Реализуйте и такой вариант. Достаточно взять $n = 3$.

## Реализуем kfold:

In [176]:
def kfold(indices, n_folds=3):
    """
    params:
        * n - objects amount in sample
        * n_folds - folds amount
    return values:
        * list with size n_folds, where every element is tuple of two 1D numpy array:
            * first array contains indices of train samples
            * second array contains indices of validation samples
    """

#     np.random.shuffle(indices)
    n = len(indices)
    size_of_one_fold = int(n / n_folds) + (n < n_folds)
    size_with_folds = size_of_one_fold * n_folds
    out_elements_amount = len(indices) - size_with_folds
    train_test_idx_list = []
    for i in range(n_folds):
        test_idx = indices[i * size_of_one_fold:(i + 1) * size_of_one_fold]
        if out_elements_amount > 0:
            test_idx = np.append(test_idx, n - out_elements_amount)
            out_elements_amount -= 1
        train_idx = np.array(list((set(indices) - set(test_idx))))
        train_test_idx_list.append((train_idx, test_idx))
    return train_test_idx_list

In [178]:
def counts_with_folds(x, n_folds=3):
    """
    x - one categorical column with type - pandas series
    y - target feature from df
    n_folds - amount of folds
    """
    
    result = x.copy()
    result.iloc[:] = 0
    for help_indx, main_indx in kfold(x.index, n_folds):
        unique_elems = np.unique(x[help_indx])
        successes_per_fold = {x : 0 for x in unique_elems}
        for unique_elem in unique_elems:
            successes_per_fold[unique_elem] = np.sum((x[help_indx] == unique_elem))
            result[np.array((x[main_indx] == unique_elem)[x[main_indx] == unique_elem].index)] = successes_per_fold[unique_elem]
    return result

In [181]:
%%time
counts_with_folds(data_cat.loc[:, 'neighbourhood'])

CPU times: user 2.96 s, sys: 0 ns, total: 2.96 s
Wall time: 2.96 s


0         109
1        1141
2        1669
3         307
4         742
         ... 
48890    2442
48891    1579
48892    1902
48893    1112
48894    1398
Name: neighbourhood, Length: 48895, dtype: int64

In [182]:
def successes_with_folds(x, y, n_folds=3):
    """
    x - one categorical column with type - pandas series
    y - target feature from df
    n_folds - amount of folds
    """
    
    result = x.copy()
    result.iloc[:] = 0
    for help_indx, main_indx in kfold(x.index, n_folds):
        unique_elems = np.unique(x[help_indx])
        successes_per_fold = {x : 0 for x in unique_elems}
        for unique_elem in unique_elems:
            successes_per_fold[unique_elem] = np.sum((x[help_indx] == unique_elem) * (y[help_indx] == 1))
            result[np.array((x[main_indx] == unique_elem)[x[main_indx] == unique_elem].index).reshape(-1)] = successes_per_fold[unique_elem]
    return result

In [207]:
successes_with_folds(data_cat.loc[:, 'neighbourhood_group'], data['target'])

0        2583
1        6546
2        6546
3        2583
4        6546
         ... 
48890    3000
48891    3000
48892    6020
48893    6020
48894    6546
Name: neighbourhood_group, Length: 48895, dtype: int64

In [184]:
def flattened_c_s_with_folds(x, y, n_folds=3):
    """
    x - one categorical column with type - pandas series
    y - target feature from df
    n_folds - amount of folds
    """
    
    result = x.copy()
    result.iloc[:] = 0
    for help_indx, main_indx in kfold(x.index, n_folds):
        unique_elems = np.unique(x[help_indx])
        successes_per_fold = {x : 0 for x in unique_elems}
        for unique_elem in unique_elems:
            successes_per_fold[unique_elem] = np.sum((x[help_indx] == unique_elem) * (y.loc[help_indx] == 1))
            result[np.array((x[main_indx] == unique_elem)[x[main_indx] == unique_elem].index)] = successes_per_fold[unique_elem]
    return result

In [185]:
flattened_c_s_with_folds(X_train_cat.loc[:, 'neighbourhood_group'], y_train_cat['target'])

39319    2002
22796    4587
26190     316
8161     4587
32473    4587
         ... 
41432    4596
8690     4596
144      1968
10987       0
12716       0
Name: neighbourhood_group, Length: 34226, dtype: int64

In [196]:
def counters_with_folds(df_cat, y):
    df_copy = df_cat.copy()
    for column in df_copy.columns:
        df_copy[column + '_counts'] = counts_with_folds(df_copy.loc[:, column])
        df_copy[column + '_successes'] = successes_with_folds(df_copy.loc[:, column], y)
        df_copy[column + '_flattened_c_s'] = flattened_c_s_with_folds(df_copy.loc[:, column], y)
        df_copy.drop(column, axis=1, inplace=True)
    return df_copy

In [197]:
counters_with_folds(X_train_cat.iloc[:50, :], y_train_cat['target'][:50])

,host_name_counts,host_name_successes,host_name_flattened_c_s,neighbourhood_group_counts,neighbourhood_group_successes,neighbourhood_group_flattened_c_s,neighbourhood_counts,neighbourhood_successes,neighbourhood_flattened_c_s,room_type_counts,room_type_successes,room_type_flattened_c_s,last_review_counts,last_review_successes,last_review_flattened_c_s
39319,0,0,0,13,4,4,0,0,0,11,0,0,0,0,0
22796,0,0,0,17,7,7,0,0,0,20,11,11,1,0,0
26190,0,0,0,3,0,0,0,0,0,11,0,0,2,1,1
8161,0,0,0,17,7,7,2,1,1,20,11,11,6,3,3
32473,0,0,0,17,7,7,0,0,0,20,11,11,0,0,0
35951,0,0,0,17,7,7,1,1,1,11,0,0,0,0,0
25954,0,0,0,3,0,0,0,0,0,20,11,11,3,0,0
18513,0,0,0,13,4,4,3,0,0,20,11,11,6,3,3
47680,0,0,0,17,7,7,1,0,0,20,11,11,6,3,3
30397,0,0,0,13,4,4,4,2,2,11,0,0,0,0,0


Посчитайте на тесте AUC-ROC метода $k$ ближайших соседей с евклидовой метрикой для выборки, где категориальные признаки заменены на счетчики. Сравните по AUC-ROC два варианта формирования выборки — с фолдингом и без. Не забудьте подобрать наилучшее число соседей $k$.

## Without folding

In [198]:
from sklearn.metrics import roc_auc_score

In [217]:
X_train_cat_without_folding = counters(X_train_cat, X_train_cat, y_train_cat['target'])

host_name
(34226,) (34226,)
(34226,) (34226,)
neighbourhood_group
(34226,) (34226,)
(34226,) (34226,)
neighbourhood
(34226,) (34226,)
(34226,) (34226,)
room_type
(34226,) (34226,)
(34226,) (34226,)
last_review
(34226,) (34226,)
(34226,) (34226,)


In [216]:
X_test_cat_without_folding = counters(X_test_cat, X_train_cat, y_train_cat['target'])

host_name
(14669,) (34226,)
(14669,) (34226,)
neighbourhood_group
(14669,) (34226,)
(14669,) (34226,)
neighbourhood
(14669,) (34226,)
(14669,) (34226,)
room_type
(14669,) (34226,)
(14669,) (34226,)
last_review
(14669,) (34226,)
(14669,) (34226,)


In [218]:
knn_regressor = KNNRegressor(10, metric='euclidean')
weights_list = ['uniform', 'distance']
k_max = 15
k_list = np.arange(k_max)
best_score_param = {'score': None, 'param': None}
for weights in weights_list:
    knn_regressor = KNNRegressor(k_max, metric='euclidean', mode=weights, test_block_size=10000)
    knn_regressor.fit(X_train_cat_without_folding, y_train_cat['target'].values)
    knn_regressor.find_kneighbors(X_test_cat_without_folding, return_distance=True)
    for k in k_list[1:]:
        knn_regressor.n_neighbors = k
        pred = knn_regressor.predict_opt(X_test_cat_without_folding)
        roc_auc = roc_auc_score(pred, y_test_cat['target'].values)
        print(f'n_neighbors: {k}, weights: {weights}, roc_auc: {roc_auc}')
        if best_score_param['score'] is None:
            best_score_param['score'] = roc_auc
            best_score_param['param'] = str(weights) + '_' + str(k)            
        elif abs(best_score_param['score'] - 0.5) < abs(roc_auc - 0.5):
            best_score_param['score'] = roc_auc
            best_score_param['param'] = str(weights) + '_' + str(k)
        
        
        
    

n_neighbors: 1, weights: uniform, roc_auc: 0.7068436985053849
n_neighbors: 2, weights: uniform, roc_auc: 0.7321806480375892
n_neighbors: 3, weights: uniform, roc_auc: 0.7384621626727605
n_neighbors: 4, weights: uniform, roc_auc: 0.7528351990184429
n_neighbors: 5, weights: uniform, roc_auc: 0.7502247936134853
n_neighbors: 6, weights: uniform, roc_auc: 0.7587909435483728
n_neighbors: 7, weights: uniform, roc_auc: 0.7578961201772032
n_neighbors: 8, weights: uniform, roc_auc: 0.7602734974407358
n_neighbors: 9, weights: uniform, roc_auc: 0.7587170536260517
n_neighbors: 10, weights: uniform, roc_auc: 0.7647457706975087
n_neighbors: 11, weights: uniform, roc_auc: 0.7631336093825913
n_neighbors: 12, weights: uniform, roc_auc: 0.7666718102399408
n_neighbors: 13, weights: uniform, roc_auc: 0.7660194216559026
n_neighbors: 14, weights: uniform, roc_auc: 0.7686026937270463
n_neighbors: 1, weights: distance, roc_auc: 0.7068436985053849
n_neighbors: 2, weights: distance, roc_auc: 0.7064461960839508
n

In [219]:
best_score_param

{'score': 0.7686026937270463, 'param': 'uniform_14'}

In [212]:
X_train_cat_with_folding = counters_with_folds(X_train_cat, y_train_cat['target'])

/home/paniquex/anaconda3/lib/python3.7/site-packages/pandas/core/computation/expressions.py:194: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  op=op_str, alt_op=unsupported[op_str]


In [213]:
knn_regressor = KNNRegressor(15, metric='euclidean')
weights_list = ['uniform', 'distance']
k_max = 15
k_list = np.arange(k_max)
best_score_param = {'score': None, 'param': None}
for weights in weights_list:
    knn_regressor = KNNRegressor(k_max, metric='euclidean', mode=weights, test_block_size=10000)
    knn_regressor.fit(X_train_cat_with_folding, y_train_cat['target'].values)
    knn_regressor.find_kneighbors(X_test_cat_without_folding, return_distance=True)
    for k in k_list[1:]:
        knn_regressor.n_neighbors = k
        pred = knn_regressor.predict_opt(X_test_cat_without_folding)
        roc_auc = roc_auc_score(pred, y_test_cat['target'].values)
        print(f'n_neighbors: {k}, weights: {weights}, roc_auc: {roc_auc}')
        if best_score_param['score'] is None:
            best_score_param['score'] = roc_auc
            best_score_param['param'] = str(weights) + '_' + str(k)            
        elif abs(best_score_param['score'] - 0.5) < abs(roc_auc - 0.5):
            best_score_param['score'] = roc_auc
            best_score_param['param'] = str(weights) + '_' + str(k)

n_neighbors: 1, weights: uniform, roc_auc: 0.6648355801476117
n_neighbors: 2, weights: uniform, roc_auc: 0.6904861775690877
n_neighbors: 3, weights: uniform, roc_auc: 0.6966272431442359
n_neighbors: 4, weights: uniform, roc_auc: 0.6982168724644027
n_neighbors: 5, weights: uniform, roc_auc: 0.7060191835758292
n_neighbors: 6, weights: uniform, roc_auc: 0.708542526594061
n_neighbors: 7, weights: uniform, roc_auc: 0.7090026851772503
n_neighbors: 8, weights: uniform, roc_auc: 0.7181746658702787
n_neighbors: 9, weights: uniform, roc_auc: 0.7254021931598083
n_neighbors: 10, weights: uniform, roc_auc: 0.7280377477498415
n_neighbors: 11, weights: uniform, roc_auc: 0.729384008185654
n_neighbors: 12, weights: uniform, roc_auc: 0.7338196526330262
n_neighbors: 13, weights: uniform, roc_auc: 0.734001592398903
n_neighbors: 14, weights: uniform, roc_auc: 0.7356151406191936
n_neighbors: 1, weights: distance, roc_auc: 0.6648355801476117
n_neighbors: 2, weights: distance, roc_auc: 0.6643526531581002
n_ne

In [215]:
best_score_param

{'score': 0.7356151406191936, 'param': 'uniform_14'}

<b>2.5 (1 балл)</b> Вернемся к задаче регрессии. Утверждается, что для задачи регрессии можно также сделать преобразование категориальных признаков в действительные числа. Для этого достаточно для каждого значения признака $f_j$ вычислить:
\begin{align}
p_j(c) = g(T_i | f_j(x_i) = c),
\end{align}

где $T_i$ - значения целевой переменной объекта $x_i$. Функция $g$ - среднее (mean) или среднеквадратичное отклонение (std).

Закодируйте категориальные признаки обоими способами и найдите значение RMSE. Используйте евклидову метрику для поиска ближайших соседей. Для какой функции $g$ значение RMSE лучше? Почему?

In [350]:
def calculate_p_j(x, y, g_func=np.mean):
    x = x.values
    y = y.values
    return g_func((x[:, None] == x) * y, axis=1)

In [351]:
data_cat['neighbourhood_group'].to_numpy()

array([1, 2, 2, ..., 2, 2, 2])

In [357]:
def calculate_p_j_for_df(df_cat, y, g_func=np.mean):
    df_copy = df_cat.copy()
    for column in df_copy.columns:
        df_copy[column + '_p_j'] = calculate_p_j(df_copy[column], y, g_func=g_func)
        df_copy.drop(column, axis=1, inplace=True)
    return df_copy

In [358]:
from sklearn.model_selection import train_test_split
X_train_cat, X_test_cat, y_train_cat, y_test_cat = train_test_split(data.drop(columns=['price',
                                                                                       'target',
                                                                                        'id',
                                                                                        'name',
                                                                                        'host_id',
                                                                                        'latitude',
                                                                                        'longitude',
                                                                                        'minimum_nights',
                                                                                        'number_of_reviews',
                                                                                        'reviews_per_month',
                                                                                        'calculated_host_listings_count',
                                                                                        'availability_365']),
                                                                                    data[['price']],
                                                                                    test_size=0.3, random_state=241)

In [359]:
X_train_cat_p_j = calculate_p_j_for_df(X_train_cat, y_train_cat['price'])

In [360]:
X_test_cat_p_j = counters(X_test_cat, X_train_cat, y_train_cat['price'])

In [ ]:
knn_regressor = KNNRegressor(15, metric='euclidean')
weights_list = ['uniform', 'distance']
k_max = 15
k_list = np.arange(k_max)
best_score_param = {'score': None, 'param': None}
for weights in weights_list:
    knn_regressor = KNNRegressor(k_max, metric='euclidean', mode=weights, test_block_size=10000)
    knn_regressor.fit(X_train_cat_p_j, y_train_cat['price'].values)
    knn_regressor.find_kneighbors(X_test_cat_p_j, return_distance=True)
    for k in k_list[1:]:
        knn_regressor.n_neighbors = k
        pred = knn_regressor.predict_opt_regression(X_test_p_j)
        roc_auc = roc_auc_score(pred, y_test_cat['price'].values)
        print(f'n_neighbors: {k}, weights: {weights}, roc_auc: {roc_auc}')
        if best_score_param['score'] is None:
            best_score_param['score'] = roc_auc
            best_score_param['param'] = str(weights) + '_' + str(k)            
        elif abs(best_score_param['score'] - 0.5) < abs(roc_auc - 0.5):
            best_score_param['score'] = roc_auc
            best_score_param['param'] = str(weights) + '_' + str(k)

In [359]:
X_train_cat_p_j = calculate_p_j_for_df(X_train_cat, y_train_cat['price'], np.std)

In [360]:
X_test_cat_p_j = counters(X_test_cat, X_train_cat, y_train_cat['price'], np.std)

In [ ]:
knn_regressor = KNNRegressor(15, metric='euclidean')
weights_list = ['uniform', 'distance']
k_max = 15
k_list = np.arange(k_max)
best_score_param = {'score': None, 'param': None}
for weights in weights_list:
    knn_regressor = KNNRegressor(k_max, metric='euclidean', mode=weights, test_block_size=10000)
    knn_regressor.fit(X_train_cat_p_j, y_train_cat['price'].values)
    knn_regressor.find_kneighbors(X_test_cat_p_j, return_distance=True)
    for k in k_list[1:]:
        knn_regressor.n_neighbors = k
        pred = knn_regressor.predict_opt_regression(X_test_p_j)
        rmse_score = rmse(pred, y_test_cat['price'].values)
        print(f'n_neighbors: {k}, weights: {weights}, rmse: {rmse_score}')
        if best_score_param['score'] is None:
            best_score_param['score'] = rmse_score
            best_score_param['param'] = str(weights) + '_' + str(k)            
        elif best_score_param['score'] > rmse_score:
            best_score_param['score'] = rmse_score
            best_score_param['param'] = str(weights) + '_' + str(k)

## Часть 3: Текстовые признаки

<b>3.1 (2 балла)</b> Перейдем от категориальным признаков к текстовым. Рассмотрим 2 способа преобразования текста в действительные числа:
- Мешок слов (Bag of Words)
- TF-IDF

[Здесь](https://scikit-learn.org/stable/modules/feature_extraction.html) вы можете прочитать про их применение в Питоне.

Сравните оба способа на задаче регресси. Какую лучше метрику использовать: евклидову или косинусную меру? Постройте графики зависимости качества решения задачи от способа преобразования, метрики и количества соседей. Мера качества - RMSE.

Объясните полученные результаты.

Перед преобразованием не забудьте уменьшить размер словаря. Например, это можно сделать за счет приведения всех слов к одному регистру и удаления [стопслов](https://en.wikipedia.org/wiki/Stop_words) (артиклей, предлогов, союзов).

## One register

In [364]:
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords_set = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/paniquex/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Delete punctuation and made one register

In [365]:
data_text = data.copy()
data_text['name'] = data_text['name'] + data_text['host_name'] + data_text['neighbourhood'] + data_text['neighbourhood_group'] + \
data_text['room_type'] + data_text['last_review']
data_text['name'] = data_text['name'].str.lower()
data_text['name'] = data_text['name'].str.split()
data_text['name'] = data_text['name'].apply(lambda x: [elem for elem in x if elem not in stopwords_set])

In [366]:
data_text['name'] = data_text['name'].apply(lambda x: [elem for elem in x if elem not in list(string.punctuation)])

In [367]:
data_text['name'] = data_text['name'].str.join(' ')

In [368]:
from sklearn.feature_extraction.text import CountVectorizer

In [403]:
vectorizer = CountVectorizer()

In [404]:
data_with_text_features_bag = vectorizer.fit_transform(data_text['name'])

In [405]:
X_train_with_text, X_test_with_text, y_train_with_text, y_test_with_text = train_test_split(data_with_text_features_bag,
                                                                                    data[['price']].values,
                                                                                    test_size=0.3, random_state=241)

In [406]:
metrics_list = ['euclidean', 'cosine']
weights_list = ['uniform', 'distance']
k_max = 15
k_list = np.arange(k_max)
best_score_param = {'score': None, 'param': None}
for metrics in metrics_list:
    for weights in weights_list:
        knn_regressor = KNNRegressor(k_max, metric=metrics, mode=weights, test_block_size=10000)
        knn_regressor.fit(X_train_with_text, y_train_with_text.reshape(-1))
        knn_regressor.find_kneighbors(X_test_with_text, return_distance=True)
        for k in k_list[1:]:
            knn_regressor.n_neighbors = k
            pred = knn_regressor.predict_opt_regression(X_test_with_text)
            rmse_score = rmse(pred, y_test_with_text.reshape(-1))
            print(f'n_neighbors: {k}, weights: {weights}, rmse: {rmse_score}')
            if best_score_param['score'] is None:
                best_score_param['score'] = rmse_score
                best_score_param['param'] = str(weights) + '_' + str(k)            
            elif best_score_param['score'] > rmse_score:
                best_score_param['score'] = rmse_score
                best_score_param['param'] = str(weights) + '_' + str(k)

MemoryError: 

In [397]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [398]:
vectorizer_tfid = TfidfVectorizer()
data_with_text_features_tfidf = vectorizer_tfid.fit_transform(data_text['name'])

In [399]:
X_train_with_text, X_test_with_text, y_train_with_text, y_test_with_text = train_test_split(data_with_text_features_tfidf,
                                                                                    data[['price']].values,
                                                                                    test_size=0.3, random_state=241)

In [402]:
metrics_list = ['euclidean', 'cosine']
weights_list = ['uniform', 'distance']
k_max = 15
k_list = np.arange(k_max)
best_score_param = {'score': None, 'param': None}
for metrics in metrics_list:
    for weights in weights_list:
        knn_regressor = KNNRegressor(k_max, metric=metrics, mode=weights, test_block_size=10000)
        knn_regressor.fit(X_train_with_text, y_train_with_text.reshape(-1))
        knn_regressor.find_kneighbors(X_test_with_text, return_distance=True)
        for k in k_list[1:]:
            knn_regressor.n_neighbors = k
            pred = knn_regressor.predict_opt_regression(X_test_with_text)
            rmse_score = rmse(pred, y_test_with_text.reshape(-1))
            print(f'n_neighbors: {k}, weights: {weights}, rmse: {rmse_score}')
            if best_score_param['score'] is None:
                best_score_param['score'] = rmse_score
                best_score_param['param'] = str(weights) + '_' + str(k)            
            elif best_score_param['score'] > rmse_score:
                best_score_param['score'] = rmse_score
                best_score_param['param'] = str(weights) + '_' + str(k)

MemoryError: 

## Collecting statistics

<b>3.2 (1 балл)</b> Используя все доступные признаки, решите задачу регрессии. Для категориальных и текстовых признаков выберите лучшие преобразования. Повлияло ли добавление количественного признака на метрику качества?

In [ ]:
# Ваш код здесь

## Часть 4: Выводы


In [ ]:
# Ваши выводы здесь (ノ°∀°)ノ⌒･*:.｡. .｡.:*･゜ﾟ･*☆